In [1]:
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
import sys

In [3]:
import tensorflow as tf
from tensorflow import keras

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))

TensorFlow version: 2.0.0


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import tensorflow as tf

import os

Using TensorFlow backend.


In [6]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [7]:
num_train = len(train_df)
y_train_init = train_df['y']
id_test = test_df['ID']

In [8]:
train_df.drop(['ID', 'y'], axis=1, inplace=True)
test_df.drop(['ID'],axis=1, inplace=True)

In [9]:
train_df.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,k,v,at,a,d,u,j,o,0,0,...,0,0,1,0,0,0,0,0,0,0
1,k,t,av,e,d,y,l,o,0,0,...,1,0,0,0,0,0,0,0,0,0
2,az,w,n,c,d,x,j,x,0,0,...,0,0,0,0,0,0,1,0,0,0
3,az,t,n,f,d,x,l,e,0,0,...,0,0,0,0,0,0,0,0,0,0
4,az,v,n,f,d,h,d,n,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
test_df.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,az,v,n,f,d,t,a,w,0,0,...,0,0,0,1,0,0,0,0,0,0
1,t,b,ai,a,d,b,g,y,0,0,...,0,0,1,0,0,0,0,0,0,0
2,az,v,as,f,d,a,j,j,0,0,...,0,0,0,1,0,0,0,0,0,0
3,az,l,n,f,d,z,l,n,0,0,...,0,0,0,1,0,0,0,0,0,0
4,w,s,as,c,d,y,i,m,0,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
test_df.isnull().values.any()

False

In [12]:
train_df.isnull().values.any()

False

In [13]:
df_all = pd.concat([train_df, test_df])

In [14]:
df_all.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,k,v,at,a,d,u,j,o,0,0,...,0,0,1,0,0,0,0,0,0,0
1,k,t,av,e,d,y,l,o,0,0,...,1,0,0,0,0,0,0,0,0,0
2,az,w,n,c,d,x,j,x,0,0,...,0,0,0,0,0,0,1,0,0,0
3,az,t,n,f,d,x,l,e,0,0,...,0,0,0,0,0,0,0,0,0,0
4,az,v,n,f,d,h,d,n,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_all = pd.get_dummies(df_all, drop_first=True)

In [16]:
df_all.head()

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X8_p,X8_q,X8_r,X8_s,X8_t,X8_u,X8_v,X8_w,X8_x,X8_y
0,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
feature_cols = [tf.feature_column.numeric_column(col) for col in df_all.columns]

In [18]:
train_df.shape

(4209, 376)

In [19]:
df_all.shape

(8418, 571)

In [20]:
X_train_init=df_all.iloc[4209:,]

In [21]:
X_train_init.shape

(4209, 571)

In [22]:
X_test=df_all.iloc[4210:,]

In [23]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val=train_test_split(X_train_init,y_train_init,test_size=0.3)

## DNN Regressor from Tensorflow

In [24]:
train_input_fn= tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train.astype(np.float32), 
                                                y= y_train.astype(np.float32), 
                                                batch_size=32, 
                                                num_epochs=200, 
                                                shuffle=True)

In [25]:
eval_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_val.astype(np.float32),
                                                      y=y_val.astype(np.float32), 
                                                      batch_size=32, 
                                                      num_epochs=1, 
                                                      shuffle=False)

In [26]:
test_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x= X_test,                                                   
                                                 batch_size=100, 
                                                 num_epochs=1, 
                                                 shuffle=False)

In [27]:
opti = tf.keras.optimizers.Adam(
    learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')

In [28]:
MODEL_DIR='E:\8K Miles\Mercedes'

In [29]:
estimator = tf.estimator.DNNRegressor(hidden_units=[128,256], feature_columns=feature_cols, optimizer=opti)
print("Training Started")
estimator.train(input_fn=train_input_fn,steps=200)
print("Training Completed")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\user\\AppData\\Local\\Temp\\tmpb8r1tg4l', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021F93888CF8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Training Started
I

In [30]:
result_eval = estimator.evaluate(input_fn=eval_input_fn)
print(pd.Series(result_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-26T22:27:59Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\user\AppData\Local\Temp\tmpb8r1tg4l\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-07-26-22:28:11
INFO:tensorflow:Saving dict for global step 200: average_loss = 168.8964, global_step = 200, label/mean = 100.17684, loss = 169.96387, prediction/mean = 98.36593
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 200: C:\Users\user\AppData\Local\Temp\tmpb8r1tg4l\model.ckpt-200
average_loss       168.896393
label/mean         100.176842
loss               169.963867
prediction/mean     98.365929
global_step        200.000000
dtype: float64


In [31]:
train_input_fn_pred= tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train.astype(np.float32), 
                                                #y= y_train.astype(np.float32), 
                                                batch_size=32, 
                                                num_epochs=1, 
                                                shuffle=False)

In [32]:
predictions_tr=[]
for pred in estimator.predict(input_fn=train_input_fn_pred):
    predictions_tr.append(np.array(pred['predictions']).astype(float))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\user\AppData\Local\Temp\tmpb8r1tg4l\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [33]:
eval_input_fn_pred = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_val.astype(np.float32),                                                      
                                                      batch_size=32, 
                                                      num_epochs=1, 
                                                      shuffle=False)

In [34]:
predictions_val=[]
for pred in estimator.predict(input_fn=eval_input_fn_pred):
    predictions_val.append(np.array(pred['predictions']).astype(float))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\user\AppData\Local\Temp\tmpb8r1tg4l\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [35]:
predictions_tr

[array([101.34779358]),
 array([104.30289459]),
 array([97.74642944]),
 array([99.09567261]),
 array([100.53772736]),
 array([98.94356537]),
 array([98.36788177]),
 array([101.78142548]),
 array([89.59762573]),
 array([96.69815826]),
 array([99.98490143]),
 array([97.43796539]),
 array([99.94235992]),
 array([96.74971771]),
 array([100.17133331]),
 array([93.50697327]),
 array([96.6448822]),
 array([100.6820755]),
 array([89.13412476]),
 array([93.09345245]),
 array([102.11304474]),
 array([97.75400543]),
 array([94.70914459]),
 array([96.86410522]),
 array([98.90995789]),
 array([99.93567657]),
 array([94.42240143]),
 array([101.13195801]),
 array([94.97940063]),
 array([97.9529953]),
 array([92.77941132]),
 array([99.70103455]),
 array([102.76705933]),
 array([97.07679749]),
 array([97.66301727]),
 array([97.63736725]),
 array([96.70677185]),
 array([98.7474823]),
 array([91.94847107]),
 array([94.72642517]),
 array([104.78481293]),
 array([97.48416138]),
 array([96.71582794]),
 arra

In [36]:
y_train

2462    110.86
6       128.76
1552    114.53
543     106.50
1253    106.03
         ...  
1920     87.17
2977     93.52
2778     94.74
3964     86.38
1372     90.96
Name: y, Length: 2946, dtype: float64

In [37]:
train_rmse = (np.square(np.array(predictions_tr[0]) - y_train)).mean(axis=0)
val_rmse = (np.square(np.array(predictions_val[0]) - y_val)).mean(axis=0)
print("RMSE Train: {}  Test: {}".format(train_rmse, val_rmse))

RMSE Train: 164.31999139780515  Test: 264.5990333008412


In [38]:
feature_cols[0].dtype

tf.float32

In [39]:
len(feature_cols)

571

In [40]:
feature_types=[feature_cols[i].dtype for i in range(len(feature_cols))] 

In [51]:
feature_cols[0].key

'X10'

In [41]:
tf.compat.v1.disable_eager_execution()

In [42]:
type('A')

str

In [44]:
inputs = {}
print(set(feature_types))
for i in range (len(feature_types)):
    inputs[feature_cols[i].key] = tf.compat.v1.placeholder(name=feature_cols[i].key, shape=[1], dtype=tf.float32)

{tf.float32}


In [45]:
serving_input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(inputs)
estimator.export_saved_model(MODEL_DIR, serving_input_fn)
print("Completed...")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Regression input must be a single string Tensor; got {'X10': <tf.Tensor 'X10:0' shape=(None,) dtype=int64>, 'X11': <tf.Tensor 'X11:0' shape=(None,) dtype=int64>, 'X12': <tf.Tensor 'X12:0' shape=(None,) dtype=int64>, 'X13': <tf.Tensor 'X13:0' shape=(None,) dt

INFO:tensorflow:'regression' : Regression input must be a single string Tensor; got {'X10': <tf.Tensor 'X10:0' shape=(None,) dtype=int64>, 'X11': <tf.Tensor 'X11:0' shape=(None,) dtype=int64>, 'X12': <tf.Tensor 'X12:0' shape=(None,) dtype=int64>, 'X13': <tf.Tensor 'X13:0' shape=(None,) dtype=int64>, 'X14': <tf.Tensor 'X14:0' shape=(None,) dtype=int64>, 'X15': <tf.Tensor 'X15:0' shape=(None,) dtype=int64>, 'X16': <tf.Tensor 'X16:0' shape=(None,) dtype=int64>, 'X17': <tf.Tensor 'X17:0' shape=(None,) dtype=int64>, 'X18': <tf.Tensor 'X18:0' shape=(None,) dtype=int64>, 'X19': <tf.Tensor 'X19:0' shape=(None,) dtype=int64>, 'X20': <tf.Tensor 'X20:0' shape=(None,) dtype=int64>, 'X21': <tf.Tensor 'X21:0' shape=(None,) dtype=int64>, 'X22': <tf.Tensor 'X22:0' shape=(None,) dtype=int64>, 'X23': <tf.Tensor 'X23:0' shape=(None,) dtype=int64>, 'X24': <tf.Tensor 'X24:0' shape=(None,) dtype=int64>, 'X26': <tf.Tensor 'X26:0' shape=(None,) dtype=int64>, 'X27': <tf.Tensor 'X27:0' shape=(None,) dtype=int64

INFO:tensorflow:Restoring parameters from C:\Users\user\AppData\Local\Temp\tmpb8r1tg4l\model.ckpt-200
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: E:\8K Miles\Mercedes\temp-b'1595783073'\saved_model.pb
Completed...


In [46]:
params = {}
params["X0"] = "a"
params["X10"] = 10
 
print(str(params))
data = json.dumps({"signature_name": "predict", "instances": [params]})

headers = {"content-type": "application/json", "Authorization": "Bearer ..."}
json_response = requests.post("<genberated from inference>:predict", data=data, headers=headers,verify=False)
json_response

predictions = json.loads(json_response.text)
predictions

{'X0': 'a', 'X10': 10}


NameError: name 'json' is not defined

In [29]:
estimator = tf.estimator.DNNRegressor(
    feature_columns=feature_columns, hidden_units=[128,256],
    optimizer=opti)


print("Training Started")
estimator.train(input_fn=train_input_fn,steps=1000)
print("Training Completed")
metrics = estimator.evaluate(input_fn=eval_input_fn)
print(pd.Series(metrics))

training_predictions = estimator.predict(input_fn=train_input_fn)
training_predictions = np.array([item['predictions'][0] for item in training_predictions])
   
validation_predictions = estimator.predict(input_fn=eval_input_fn)
validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])


train_rmse = (np.square(training_predictions - y_train)).mean(axis=0)
test_rmse = (np.square(validation_predictions - y_eval)).mean(axis=0)
print("RMSE Train: {}  Test: {}".format(train_rmse, test_rmse))
"""
feature_spec = tf.feature_column.make_parse_example_spec(feature_columns)
serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
estimator.export_saved_model(MODEL_DIR, serving_input_receiver_fn)
"""
inputs = {}
inputs['X10'] = tf.placeholder(name='X10', shape=[1], dtype=tf.float32)
inputs['X0'] = tf.placeholder(name='X0', shape=[1], dtype=tf.string)
        
serving_input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(inputs)
estimator.export_saved_model(MODEL_DIR, serving_input_fn)


print("Completed...")

params = {}
params["X0"] = "a"
params["X10"] = 10
print(str(params))
data = json.dumps({"signature_name": "predict", "instances": [params]})


headers = {"content-type": "application/json", "Authorization": "Bearer ..."}
json_response = requests.post("<genberated from inference>:predict", 
data=data, headers=headers,verify=False)
json_response

predictions = json.loads(json_response.text)
predictions

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\user\\AppData\\Local\\Temp\\tmp6p2rjnol', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000016213795748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Training Started
I

ValueError: operands could not be broadcast together with shapes (589200,) (2946,) 